importar librerías 

In [9]:
import time

import requests
from bs4 import BeautifulSoup
import pandas as pd
import sqlite3

request de la url que se nos pone a disposición 

In [10]:
url = "https://www.macrotrends.net/stocks/charts/TSLA/tesla/revenue "
html_data = requests.get(url, time.sleep(10)).text

In [11]:
soup = BeautifulSoup(html_data,"html.parser")

In [14]:
#find all tables
tables = soup.find_all('table')

for index,table in enumerate(tables):
    if ("Tesla Quarterly Revenue" in str(table)):
        table_index = index

#create a dataframe        
tesla_revenue = pd.DataFrame(columns=["Date", "Revenue"])

for row in tables[table_index].tbody.find_all("tr"):
    col = row.find_all("td")
    if (col != []):
        Date = col[0].text
        Revenue = col[1].text.replace("$", "").replace(",", "")
        tesla_revenue = tesla_revenue.append({"Date":Date, "Revenue":Revenue}, ignore_index=True)


NameError: name 'table_index' is not defined

In [15]:
import pandas as pd
from urllib.error import HTTPError
import time

# Define the URL and other variables
url = "your_url_here"
max_retries = 3
retry_delay = 5  # seconds

# Retry mechanism
for _ in range(max_retries):
    try:
        tables = pd.read_html(url, match="Tesla Quarterly Revenue", flavor='bs4')
        if tables:
            tesla_revenue = pd.DataFrame(columns=["Date", "Revenue"])
            table_index = 0  # Define the appropriate table index
            for row in tables[table_index].tbody.find_all("tr"):
                col = row.find_all("td")
                if col:
                    date = col[0].text.strip()
                    revenue = col[1].text.strip().replace(',','').replace('$','')
                    tesla_revenue = tesla_revenue.append({"Date": date, "Revenue": revenue}, ignore_index=True)
            break  # Break the loop if successful
        else:
            print("No tables found on the page.")
            break  # Break the loop if no tables found
    except HTTPError as e:
        if e.code == 403:
            print("Received HTTP 403 error. Retrying in {} seconds...".format(retry_delay))
            time.sleep(retry_delay)
        else:
            raise  # Re-raise other HTTP errors


ValueError: No tables found

In [12]:
#using read_html

tesla_revenue=pd.read_html(url, match="Tesla Quarterly Revenue", flavor='bs4')[0]
tesla_revenue = tesla_revenue.rename(columns={"Tesla Quarterly Revenue(Millions of US $)":"Date","Tesla Quarterly Revenue(Millions of US $).1":"Revenue"}) #Rename df columns to 'Date' and 'Revenue'
tesla_revenue["Revenue"] = tesla_revenue['Revenue'].str.replace(',|\$',"") # remove the comma and dollar sign from the 'Revenue' column
tesla_revenue.head()

HTTPError: HTTP Error 403: Forbidden

In [8]:
import pandas as pd
from urllib.error import HTTPError  # Import the HTTPError class from urllib.error module
import time

# Your retry mechanism code
max_retries = 3
retry_delay = 5  # seconds

for _ in range(max_retries):
    try:
        tesla_revenue = pd.read_html(url, match="Tesla Quarterly Revenue", flavor='bs4')[0]
        break  # Break the loop if successful
    except HTTPError as e:
        if e.code == 403:
            print("Received HTTP 403 error. Retrying in {} seconds...".format(retry_delay))
            time.sleep(retry_delay)
        else:
            raise  # Re-raise other HTTP errors


Received HTTP 403 error. Retrying in 5 seconds...
Received HTTP 403 error. Retrying in 5 seconds...
Received HTTP 403 error. Retrying in 5 seconds...


In [6]:
tesla_revenue = tesla_revenue[tesla_revenue['Revenue'] != ""]
tesla_revenue

,Date,Revenue


In [7]:
type(tesla_revenue)

pandas.core.frame.DataFrame

In [8]:
records = tesla_revenue.to_records(index=False)
list_of_tuples = list(records)
list_of_tuples

[('2023-03-31', '23329'),
 ('2022-12-31', '24318'),
 ('2022-09-30', '21454'),
 ('2022-06-30', '16934'),
 ('2022-03-31', '18756'),
 ('2021-12-31', '17719'),
 ('2021-09-30', '13757'),
 ('2021-06-30', '11958'),
 ('2021-03-31', '10389'),
 ('2020-12-31', '10744'),
 ('2020-09-30', '8771'),
 ('2020-06-30', '6036'),
 ('2020-03-31', '5985'),
 ('2019-12-31', '7384'),
 ('2019-09-30', '6303'),
 ('2019-06-30', '6350'),
 ('2019-03-31', '4541'),
 ('2018-12-31', '7226'),
 ('2018-09-30', '6824'),
 ('2018-06-30', '4002'),
 ('2018-03-31', '3409'),
 ('2017-12-31', '3288'),
 ('2017-09-30', '2985'),
 ('2017-06-30', '2790'),
 ('2017-03-31', '2696'),
 ('2016-12-31', '2285'),
 ('2016-09-30', '2298'),
 ('2016-06-30', '1270'),
 ('2016-03-31', '1147'),
 ('2015-12-31', '1214'),
 ('2015-09-30', '937'),
 ('2015-06-30', '955'),
 ('2015-03-31', '940'),
 ('2014-12-31', '957'),
 ('2014-09-30', '852'),
 ('2014-06-30', '769'),
 ('2014-03-31', '621'),
 ('2013-12-31', '615'),
 ('2013-09-30', '431'),
 ('2013-06-30', '405'),


In [11]:
connection = sqlite3.connect('Tesla.db')

In [12]:
c = connection.cursor()

# Create table
c.execute('''CREATE TABLE revenue
             (Date, Revenue)''')

In [13]:
# Insert the values
c.executemany('INSERT INTO revenue VALUES (?,?)', list_of_tuples)
# Save (commit) the changes
connection.commit()

In [14]:
for row in c.execute('SELECT * FROM revenue'):
    print(row)

('2023-03-31', '23329')
('2022-12-31', '24318')
('2022-09-30', '21454')
('2022-06-30', '16934')
('2022-03-31', '18756')
('2021-12-31', '17719')
('2021-09-30', '13757')
('2021-06-30', '11958')
('2021-03-31', '10389')
('2020-12-31', '10744')
('2020-09-30', '8771')
('2020-06-30', '6036')
('2020-03-31', '5985')
('2019-12-31', '7384')
('2019-09-30', '6303')
('2019-06-30', '6350')
('2019-03-31', '4541')
('2018-12-31', '7226')
('2018-09-30', '6824')
('2018-06-30', '4002')
('2018-03-31', '3409')
('2017-12-31', '3288')
('2017-09-30', '2985')
('2017-06-30', '2790')
('2017-03-31', '2696')
('2016-12-31', '2285')
('2016-09-30', '2298')
('2016-06-30', '1270')
('2016-03-31', '1147')
('2015-12-31', '1214')
('2015-09-30', '937')
('2015-06-30', '955')
('2015-03-31', '940')
('2014-12-31', '957')
('2014-09-30', '852')
('2014-06-30', '769')
('2014-03-31', '621')
('2013-12-31', '615')
('2013-09-30', '431')
('2013-06-30', '405')
('2013-03-31', '562')
('2012-12-31', '306')
('2012-09-30', '50')
('2012-06-30', 